## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 13)

## 訓練データとテストデータに分割

In [4]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [5]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

## 訓練データで訓練

In [6]:
XGB = XGBRegressor()

In [7]:
XGB = XGB.fit(train_X, train_y)

[23:00:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
XGB.feature_importances_

array([0.19221137, 0.14225699, 0.2762786 , 0.00638812, 0.02264222,
       0.01574961, 0.0107585 , 0.00215186, 0.00496602, 0.23729466,
       0.08930208], dtype=float32)

In [10]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, [2, 3, 4, 5, 7, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.276, 'beds'),
 (0.192, 'bathrooms'),
 (0.142, 'bedrooms'),
 (0.023, 'cancellation_policy_moderate'),
 (0.016, 'cancellation_policy_strict'),
 (0.011, 'cancellation_policy_super_strict_30'),
 (0.006, 'number_of_reviews'),
 (0.005, 'room_type_Private room'),
 (0.002, 'cancellation_policy_super_strict_60')]

## テストデータで実行

In [11]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [12]:
test_x.shape, test_y.shape

((11065, 11), (11065,))

In [13]:
pred_y = XGB.predict(test_x)

In [14]:
np.sqrt(mean_squared_error(test_y, pred_y))

130.80940224417495

## 検証データで実行

In [15]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [16]:
valid.shape

(18528, 12)

In [17]:
# ID の保存
valid_pass = valid.id.values

In [18]:
valid_X = valid.iloc[:, 1:].values

In [19]:
valid_X.shape, train_X.shape

((18528, 11), (44258, 11))

In [20]:
# valid_X.head()

In [21]:
pred_valid_y = XGB.predict(valid_X)

In [22]:
pred_valid_y.shape

(18528,)

In [23]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [24]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [25]:
result_df.to_csv("./XGB_7.csv", header=False)